In [7]:
import pandas as pd

from dotenv import load_dotenv
load_dotenv()


True

In [8]:
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange,
    Dimension,
    Metric,
    RunReportRequest,
)

In [9]:
PROPERTY_ID = 386396252

In [10]:

def run_report(property_id=PROPERTY_ID):

    client = BetaAnalyticsDataClient()

    request = RunReportRequest(
        property=f"properties/{property_id}",
        dimensions=[
                Dimension(name="date"),
                Dimension(name="city")
            ],
        metrics=[
                Metric(name="activeUsers"),
                Metric(name="sessions"),
                Metric(name="newUsers"),
            ],
        date_ranges=[
            DateRange(start_date="2020-03-31", end_date="today")
            ],
    )

    response = client.run_report(request)

    return response



In [11]:
def parse_report(res):

    dim_headers = [d.name for d in res.dimension_headers]
    metric_headers = [m.name for m in res.metric_headers]
    headers = dim_headers + metric_headers

    data = []

    for row in res.rows:
        dims = [d.value for d in row.dimension_values]
        metrics = [m.value for m in row.metric_values]
        data.append(dims + metrics)

    df = pd.DataFrame(data, columns=headers)

    return df


In [13]:
res = run_report()
df = parse_report(res)

df.head()

,date,city,activeUsers,sessions,newUsers
0,20231030,Moscow,61,88,54
1,20231024,Moscow,60,87,54
2,20231120,Moscow,60,86,55
3,20231101,Moscow,59,88,55
4,20231116,Moscow,58,90,55


In [14]:
import gspread

gc = gspread.service_account('credentials.json')

In [15]:

wks = gc.open("gspread_test_123").sheet1
wks.update([df.columns.values.tolist()] + df.values.tolist())

/var/folders/s_/t8ytmm9940j564xd698kgmm00000gn/T/ipykernel_29686/2213771718.py:2: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  wks.update([df.columns.values.tolist()] + df.values.tolist())


{'spreadsheetId': '1N6qEd_HFBQ_dQqQwcZsbfU2Q6lKcmKOBcQs0FWODCEA',
 'updatedRange': 'Sheet1!A1:E10001',
 'updatedRows': 10001,
 'updatedColumns': 5,
 'updatedCells': 50005}